##Leyendo datos

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Books.csv

--2022-01-21 17:54:41--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Books.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2140933459 (2.0G) [application/octet-stream]
Saving to: ‘Books.csv’

Books.csv           100%[===================>]   1.99G  18.4MB/s    in 1m 52s  

2022-01-21 17:56:34 (18.2 MB/s) - ‘Books.csv’ saved [2140933459/2140933459]



In [3]:
!pip install numpy
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 11.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623232 sha256=4f1a58f79dda16ce8437ebe722dc9a11ac0174f5493958b7d9d35b52b058c7d7
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [4]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans
from sklearn.model_selection import train_test_split

In [5]:
reviews = pd.read_csv("Books.csv")

In [6]:
#item,user,rating,timestamp
reviews = reviews.rename(columns = {"0001713353":"item",	"A1C6M8LCIX4M6M" : "user", 	"5.0" : "rating", "1123804800": "timestamp"})
print(reviews.shape)
reviews.head()

(51311620, 4)


,item,user,rating,timestamp
0,0001713353,A1REUF3A1YCPHM,5.0,1112140800
1,0001713353,A1YRBRK2XM5D5,5.0,1081036800
2,0001713353,A1V8ZR5P78P4ZU,5.0,1077321600
3,0001713353,A2ZB06582NXCIV,5.0,1475452800
4,0001713353,ACPQVNRD3Z09X,5.0,1469750400


##Preprocesando datos

In [7]:
min_book_ratings = 20
max_book_ratings = 50
filter_books = (reviews['item'].value_counts() > min_book_ratings) & (reviews['item'].value_counts() < max_book_ratings)
filter_books = filter_books[filter_books].index.tolist()


min_user_ratings = 20
max_user_ratings = 50
filter_users = (reviews['user'].value_counts() > min_user_ratings) & (reviews['user'].value_counts() < max_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

df_new = reviews[(reviews['item'].isin(filter_books)) & (reviews['user'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(reviews.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(51311620, 4)
The new data frame shape:	(754962, 4)


In [46]:
sample = df_new.sample(frac = 0.20)
print(sample.shape)

(150992, 4)


In [9]:
"""
byuser = reviews.groupby("user").count().sort_values("item", ascending = False)
print(byuser.shape)
byuser.head()

has20ratings = byuser["item"] >=50
users20ratings = byuser[has20ratings]
print(users20ratings.shape)
users20ratings.head()
print(users20ratings.columns.tolist())
userIDs = users20ratings.index
userIDs = userIDs.tolist()
print(userIDs)
"""

'\nbyuser = reviews.groupby("user").count().sort_values("item", ascending = False)\nprint(byuser.shape)\nbyuser.head()\n\nhas20ratings = byuser["item"] >=50\nusers20ratings = byuser[has20ratings]\nprint(users20ratings.shape)\nusers20ratings.head()\nprint(users20ratings.columns.tolist())\nuserIDs = users20ratings.index\nuserIDs = userIDs.tolist()\nprint(userIDs)\n'

In [10]:
"""
has20ratings = (byuser["item"] >=20) & (byuser["item"] <= 50)
users20ratings = byuser[has20ratings]
print(users20ratings.shape)
users20ratings.head()
print(users20ratings.columns.tolist())
userIDs = users20ratings.index
userIDs = userIDs.tolist()
print(userIDs)
"""

'\nhas20ratings = (byuser["item"] >=20) & (byuser["item"] <= 50)\nusers20ratings = byuser[has20ratings]\nprint(users20ratings.shape)\nusers20ratings.head()\nprint(users20ratings.columns.tolist())\nuserIDs = users20ratings.index\nuserIDs = userIDs.tolist()\nprint(userIDs)\n'

In [11]:
"""
boolean_series = reviews["user"].isin(userIDs)
user_ratings = reviews[boolean_series]
print(user_ratings.shape)
user_ratings.head()
"""

'\nboolean_series = reviews["user"].isin(userIDs)\nuser_ratings = reviews[boolean_series]\nprint(user_ratings.shape)\nuser_ratings.head()\n'

In [47]:
df = sample[["item", "user", "rating"]]
df.head()

,item,user,rating
31426367,1590595823,AQWPMPDM0M08L,4.0
1342906,006078220X,A9TLKYVZ7Y0OX,5.0
43683788,1118615689,AHCVVNLI56OI9,5.0
39512700,0393083012,A1X7G4QPJ1S1V5,3.0
32430605,161159958X,A2S0JNPCRHN5D8,5.0


##Tran test split

In [ ]:
train, test= train_test_split(df)

In [ ]:
type(test)

In [ ]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(train, reader)
#train, test = train_test_split(data)

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

##KNN

In [ ]:
# recommender
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [ ]:
trainingSet = data.build_full_trainset()

algo.fit(trainingSet)

In [ ]:
print(trainingSet)

In [ ]:
#test_data = Dataset.load_from_df(test, reader)

In [ ]:
#algo.test(test)

In [ ]:
prediction = algo.predict("AMZ13TUQHOIEY", "B00J3VRTHE")
print(prediction.est)
# 4.0

## Agregar datos de goodreads

https://nijianmo.github.io/amazon/index.html#:~:text=core%20(27%2C164%2C983%20reviews)-,ratings%20only,-(51%2C311%2C621%20ratings)

In [ ]:
##ALS

##ALS

In [13]:
from surprise import BaselineOnly
from surprise.model_selection import cross_validate


In [48]:
df = sample[["item", "user", "rating"]]
#dataset.load
#build train set
# split

In [24]:
#
#trainset, testset = train_test_split(df, test_size = 0.25)

In [49]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df, reader)

In [56]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo_ALS = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo_ALS, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'fit_time': (0.5695855617523193, 0.533804178237915, 0.6279246807098389),
 'test_rmse': array([0.98500918, 0.99800584, 0.99619687]),
 'test_time': (0.3907926082611084, 0.6853547096252441, 0.3967440128326416)}

In [57]:
algo_ALS.predict("AQWPMPDM0M08L", 1590595823)

Prediction(uid='AQWPMPDM0M08L', iid=1590595823, r_ui=None, est=4.381275953189883, details={'was_impossible': False})

In [52]:
df.head()

,item,user,rating
31426367,1590595823,AQWPMPDM0M08L,4.0
1342906,006078220X,A9TLKYVZ7Y0OX,5.0
43683788,1118615689,AHCVVNLI56OI9,5.0
39512700,0393083012,A1X7G4QPJ1S1V5,3.0
32430605,161159958X,A2S0JNPCRHN5D8,5.0


In [53]:
sample.info

<bound method DataFrame.info of                 item            user  rating   timestamp
40347344  0590878557  A3BO7YGUVCYECE     5.0  1393977600
43626975  0999173626  A3JOESJ8ZNEEHT     4.0  1513468800
40057646  0495901172   ALNRC5K40NY1W     5.0  1480032000
4964990   0226534219   AEDR28X7F3WWX     2.0  1429920000
22593406  1419349473  A3F7F7QKQP2FKT     5.0  1353888000
...              ...             ...     ...         ...
29523341  1521391408  A29MTZOXP3BLVE     5.0  1495670400
14248351  0684831988   AB4UAD9KGLJPN     4.0   903830400
4895469   0199550166  A2C7JDP9PUMNMR     5.0  1325376000
38485103  0140270361   AA9L4LSV4UUB6     5.0  1425945600
41969020  0811877817  A3F7NBKM0NWLHR     5.0  1416096000

[15099 rows x 4 columns]>

In [43]:

from surprise import Dataset

from surprise.accuracy import rmse
from surprise import accuracy
from surprise import SVD
from surprise.model_selection import KFold

In [28]:
type(data)

surprise.dataset.DatasetAutoFolds

In [51]:
kf = KFold(n_splits=3)

algo_ALS = BaselineOnly(bsl_options=bsl_options)

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo_ALS.fit(trainset)
    predictions = algo_ALS.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)


Estimating biases using als...
RMSE: 0.9972
Estimating biases using als...
RMSE: 0.9929
Estimating biases using als...
RMSE: 0.9905


In [58]:
trainset = data.build_full_trainset()
algo_ALS = BaselineOnly(bsl_options=bsl_options)
predictions = algo_ALS.fit(trainset)
#accuracy.rmse(predictions)

Estimating biases using als...


In [59]:
predictions.predict("AQWPMPDM0M08L", 1590595823)

Prediction(uid='AQWPMPDM0M08L', iid=1590595823, r_ui=None, est=4.382099713892127, details={'was_impossible': False})

In [60]:
predictions.predict("A9TLKYVZ7Y0OX", "006078220X")

Prediction(uid='A9TLKYVZ7Y0OX', iid='006078220X', r_ui=None, est=4.382099713892127, details={'was_impossible': False})

In [42]:
t = predictions.test(testset[0])

KeyError: ignored

In [41]:
testset.head

<bound method NDFrame.head of                 item            user  rating
32093201  1603742107   AI7WD0QY4T1VI     5.0
9854152   0393608441   A81HGJ7GB7VK7     4.0
4098352   0142002879  A23K4L5YV6G9Y5     3.0
17485414  0805054464  A36JMENJWF3SEV     3.0
32791764  1619213494  A2OLJCTP892JFU     5.0
...              ...             ...     ...
43885135  0985467207   AEW44FGGVPEYV     3.0
36466503  0981084303   AO7199A84NHWW     5.0
42890599  0964894238  A28LTJZ3TXD58U     5.0
47342795  1623658837  A31VEK359K5LCP     4.0
4785638   0152045708  A1JW7M9QIIZN8A     3.0

[3775 rows x 3 columns]>

method

In [ ]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]